In [1]:
import csv
import torch
import evaluate

from tqdm.auto import tqdm
from torch.optim import AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler

/home/jsh/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "classla/xlm-roberta-base-multilingual-text-genre-classifier"

In [3]:
datasets = DatasetDict()

datasets["train"] = load_dataset("csv", data_files="./data/train.csv")["train"]
datasets["valid"] = load_dataset("csv", data_files="./data/valid.csv")["train"]

datasets

Found cached dataset csv (/Users/intstr/.cache/huggingface/datasets/csv/default-cedb9b3bf563c2d8/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset csv (/Users/intstr/.cache/huggingface/datasets/csv/default-5da9b9027cdd7c99/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 799997
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 100000
    })
})

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=512, truncation=True, padding="max_length")

tokenized_datasets = datasets.map(tokenize_function, batched=True)

Loading cached processed dataset at /Users/intstr/.cache/huggingface/datasets/csv/default-cedb9b3bf563c2d8/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-955973082d65fd93.arrow
Loading cached processed dataset at /Users/intstr/.cache/huggingface/datasets/csv/default-5da9b9027cdd7c99/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-8ade8154317d22a9.arrow


In [5]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=77).select(range(1000))
small_valid_dataset = tokenized_datasets["valid"].shuffle(seed=77).select(range(1000))

Loading cached shuffled indices for dataset at /Users/intstr/.cache/huggingface/datasets/csv/default-cedb9b3bf563c2d8/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-41735d33763bc5e7.arrow
Loading cached shuffled indices for dataset at /Users/intstr/.cache/huggingface/datasets/csv/default-5da9b9027cdd7c99/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-45daeb817e47c91a.arrow


In [7]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=16)
valid_dataloader = DataLoader(small_valid_dataset, batch_size=16)

In [8]:
with open("./data/id_to_label.csv", "r") as f:
    reader = csv.DictReader(f)
    id2label = {row["id"]: row["label"] for row in reader}
label2id = {label: id for id, label in id2label.items()}

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=18, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at classla/xlm-roberta-base-multilingual-text-genre-classifier and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([18]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([18, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [11]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [12]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
metric = evaluate.load("accuracy")
model.eval()
for batch in valid_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()